# ROI Analysis Verification

This notebook walks through the gamma-ray isotope ROI (Region of Interest) analysis step by step.
It verifies that channel-based ROI windows are correctly aligned to actual spectral peaks,
prototypes energy-based ROI definitions, and demonstrates the data gap NaN fix.

**Sections:**
1. Setup and Imports
2. Load Calibration
3. Load a Raw Spectrum from CNF File
4. Load Current Channel-Based ROIs and Overlay on Spectrum
5. Define Energy-Based ROI Definitions
6. Energy-to-Channel Conversion
7. Step-by-Step Net Count Extraction
8. Compare Channel-Based vs Energy-Based Results
9. Calibration Drift Robustness Demo
10. Time Series and Data Gap Visualization
11. Proposed Energy ROI File Format

## Section 1: Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import h5py
import sys
import os

# Add paths so we can import project modules
sys.path.insert(0, '.')
sys.path.insert(0, '..')

from cnf_parser_standalone import read_cnf
from spectrum_calibration import read_calibration_file, apply_calibration
from spectra_utils import parse_roi
import sample_collection

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (14, 6)

print('All imports successful.')

## Section 2: Load Calibration

Load the single source-of-truth calibration from the consolidated location.

In [ ]:
cal = read_calibration_file('calibration/calibration_coefficients.txt')
c0, c1, c2, c3 = cal

print('Calibration coefficients:')
print(f'  c0 = {c0}')
print(f'  c1 = {c1}')
print(f'  c2 = {c2}')
print(f'  c3 = {c3}')
print()
print(f'Calibration equation: E(keV) = {c0:.6f} + {c1:.9f} * channel')

## Section 3: Load a Raw Spectrum from CNF File

Load a test CNF file and plot the full spectrum on the calibrated energy axis.

In [ ]:
data = read_cnf('../test/2026-02-17_19-30-36-434509.CNF', verbose=True)
counts = np.array(data['counts'], dtype=float)
n_channels = len(counts)
print(f'Loaded {n_channels} channels, total counts: {np.sum(counts):.0f}')

In [ ]:
# Apply calibration to get energy axis
energies = apply_calibration(counts, cal)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

ax1.plot(energies, counts, 'b-', linewidth=0.5)
ax1.set_xlabel('Energy (keV)')
ax1.set_ylabel('Counts')
ax1.set_title('Full Spectrum (linear scale)')
ax1.set_xlim(0, 3000)
ax1.grid(True, alpha=0.3)

ax2.semilogy(energies, counts + 1, 'b-', linewidth=0.5)
ax2.set_xlabel('Energy (keV)')
ax2.set_ylabel('Counts (log)')
ax2.set_title('Full Spectrum (log scale)')
ax2.set_xlim(0, 3000)
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.show()

print()
print('NOTE on indexing discrepancy:')
print('  apply_calibration() uses 0-indexed channels: E = c0 + c1 * np.arange(len(counts))')
print('  h5_analysis.py line 110 uses 1-indexed: E = (x+1)*c1 + c0')
print('  This causes a ~0.33 keV offset between the two approaches.')

## Section 4: Load Current Channel-Based ROIs and Overlay on Spectrum

This is the **key verification step** — visually check if the channel-based ROI windows
are correctly aligned with the actual spectral peaks.

In [ ]:
rois = parse_roi('analysis/roi_simple.dat')

print(f'Loaded {len(rois)} ROIs:')
print(f'{"Isotope":<10} {"Energy(keV)":<14} {"Peak Ch":<14} {"Peak E(keV)":<18} {"Offset(keV)":<14} {"Origin"}')
print('-' * 90)
for r in rois:
    peak_e_lo = c0 + c1 * r.peak[0]
    peak_e_hi = c0 + c1 * r.peak[1]
    peak_e_center = (peak_e_lo + peak_e_hi) / 2
    offset = peak_e_center - r.energy
    print(f'{r.isotope:<10} {r.energy:<14.2f} {str(r.peak):<14} {peak_e_lo:.1f}-{peak_e_hi:.1f}  '
          f'  {offset:>+8.2f}       {r.origin}')

In [ ]:
# Plot spectrum with ROI overlays
colors = plt.cm.Set1(np.linspace(0, 1, len(rois)))

fig, axes = plt.subplots(len(rois), 1, figsize=(14, 4 * len(rois)))

for i, r in enumerate(rois):
    ax = axes[i]
    
    # Determine plot range: include background sidebands with margin
    all_channels = [r.peak[0], r.peak[1]]
    for bkg in r.bkg:
        all_channels.extend([bkg[0], bkg[1]])
    ch_lo = min(all_channels) - 30
    ch_hi = max(all_channels) + 30
    ch_lo = max(0, ch_lo)
    ch_hi = min(n_channels - 1, ch_hi)
    
    e_lo = c0 + c1 * ch_lo
    e_hi = c0 + c1 * ch_hi
    
    # Plot spectrum in this range
    mask = (energies >= e_lo) & (energies <= e_hi)
    ax.plot(energies[mask], counts[mask], 'b-', linewidth=1)
    
    # Shade peak window (solid)
    peak_e_lo = c0 + c1 * r.peak[0]
    peak_e_hi = c0 + c1 * r.peak[1]
    ax.axvspan(peak_e_lo, peak_e_hi, alpha=0.3, color=colors[i], label='Peak window')
    
    # Shade background sidebands (hatched)
    for j, bkg in enumerate(r.bkg):
        bkg_e_lo = c0 + c1 * bkg[0]
        bkg_e_hi = c0 + c1 * bkg[1]
        ax.axvspan(bkg_e_lo, bkg_e_hi, alpha=0.15, color='gray',
                   hatch='///', label='Background' if j == 0 else None)
    
    # Mark expected energy
    ax.axvline(r.energy, color='red', linestyle='--', alpha=0.7, label=f'Expected {r.energy} keV')
    
    ax.set_xlabel('Energy (keV)')
    ax.set_ylabel('Counts')
    ax.set_title(f'{r.isotope} ({r.energy} keV) - {r.origin}')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Section 5: Define Energy-Based ROI Definitions

Instead of hardcoded channel numbers, define ROIs in keV.
These are derived from the current channel-based ROIs + calibration,
then manually adjusted to be properly centered on peaks.

In [ ]:
# Derive initial energy values from current channel-based ROIs
print('Current ROIs converted to energy:')
print(f'{"Isotope":<10} {"Peak keV range":<20} {"Bkg keV ranges"}')
print('-' * 80)
for r in rois:
    peak_lo = c0 + c1 * r.peak[0]
    peak_hi = c0 + c1 * r.peak[1]
    bkg_strs = []
    for bkg in r.bkg:
        bkg_lo = c0 + c1 * bkg[0]
        bkg_hi = c0 + c1 * bkg[1]
        bkg_strs.append(f'({bkg_lo:.1f}, {bkg_hi:.1f})')
    print(f'{r.isotope:<10} ({peak_lo:.1f}, {peak_hi:.1f})    {"  ".join(bkg_strs)}')

In [ ]:
# Energy-based ROI definitions (keV)
# Derived from channel-based ROIs and adjusted to center on known peak energies
energy_rois = [
    {'isotope': 'Pb214', 'energy': 351.71,
     'peak_keV': (348.0, 355.0),
     'bkg_keV': [(340.0, 346.0), (357.0, 363.0)],
     'origin': 'Naturally Occurring (U238)'},

    {'isotope': 'Bi214', 'energy': 608.7,
     'peak_keV': (605.0, 613.0),
     'bkg_keV': [(596.0, 603.0), (615.0, 622.0)],
     'origin': 'Naturally Occurring (U238)'},

    {'isotope': 'Pb212', 'energy': 238.6,
     'peak_keV': (235.0, 241.0),
     'bkg_keV': [(228.0, 233.0)],
     'origin': 'Naturally Occurring (Th232)'},

    {'isotope': 'Tl208', 'energy': 2612.5,
     'peak_keV': (2608.0, 2618.0),
     'bkg_keV': [(2596.0, 2605.0), (2620.0, 2630.0)],
     'origin': 'Naturally Occurring (Th232)'},

    {'isotope': 'K40', 'energy': 1459.41,
     'peak_keV': (1455.0, 1465.0),
     'bkg_keV': [(1443.0, 1452.0), (1468.0, 1478.0)],
     'origin': 'Naturally Occurring'},

    {'isotope': 'Cs134', 'energy': 604.0,
     'peak_keV': (601.0, 606.0),
     'bkg_keV': [(594.0, 599.0)],
     'origin': 'Reactor'},

    {'isotope': 'Cs137', 'energy': 661.7,
     'peak_keV': (658.0, 663.0),
     'bkg_keV': [(650.0, 656.0), (666.0, 672.0)],
     'origin': 'Technically Enhanced/Naturally Occurring'},
]

print(f'Defined {len(energy_rois)} energy-based ROIs:')
for roi in energy_rois:
    print(f"  {roi['isotope']}: {roi['energy']} keV, peak window {roi['peak_keV']}")

## Section 6: Energy-to-Channel Conversion

Define helper functions that invert the calibration equation to convert keV back to channels.

In [ ]:
def energy_to_channel(energy_keV, calibration):
    """Convert energy (keV) to channel number.
    Inverts E = c0 + c1*ch  =>  ch = (E - c0) / c1
    Only valid for linear calibration (c2=c3=0)."""
    c0, c1, c2, c3 = calibration
    if abs(c2) > 1e-10 or abs(c3) > 1e-15:
        raise ValueError('energy_to_channel only supports linear calibration (c2=c3=0)')
    return (energy_keV - c0) / c1


def energy_roi_to_channels(roi_def, calibration):
    """Convert an energy-based ROI definition to channel indices.
    Returns a dict with 'peak' and 'bkg' as integer channel ranges."""
    peak_lo = int(np.round(energy_to_channel(roi_def['peak_keV'][0], calibration)))
    peak_hi = int(np.round(energy_to_channel(roi_def['peak_keV'][1], calibration)))
    bkg_channels = []
    for bkg_lo_keV, bkg_hi_keV in roi_def['bkg_keV']:
        bkg_lo = int(np.round(energy_to_channel(bkg_lo_keV, calibration)))
        bkg_hi = int(np.round(energy_to_channel(bkg_hi_keV, calibration)))
        bkg_channels.append([bkg_lo, bkg_hi])
    return {'peak': [peak_lo, peak_hi], 'bkg': bkg_channels}


# Round-trip validation: convert current channel ROIs to energy and back
print('Round-trip validation (channel -> energy -> channel):')
print(f'{"Isotope":<10} {"Original peak":<16} {"Recovered peak":<16} {"Match?"}')
print('-' * 60)
for r in rois:
    # Forward: channel -> energy
    e_lo = c0 + c1 * r.peak[0]
    e_hi = c0 + c1 * r.peak[1]
    # Backward: energy -> channel
    ch_lo_recovered = int(np.round(energy_to_channel(e_lo, cal)))
    ch_hi_recovered = int(np.round(energy_to_channel(e_hi, cal)))
    match = (ch_lo_recovered == r.peak[0]) and (ch_hi_recovered == r.peak[1])
    print(f'{r.isotope:<10} {str(r.peak):<16} [{ch_lo_recovered}, {ch_hi_recovered}]     {"OK" if match else "MISMATCH"}')

## Section 7: Step-by-Step Net Count Extraction

Walk through the net count calculation for K-40 (1461 keV) — a strong, easy-to-see peak.

In [ ]:
# Pick K40 from energy_rois
k40_energy_roi = [r for r in energy_rois if r['isotope'] == 'K40'][0]
k40_channels = energy_roi_to_channels(k40_energy_roi, cal)

print('K-40 Energy-based ROI:')
print(f'  Peak window: {k40_energy_roi["peak_keV"]} keV  ->  channels {k40_channels["peak"]}')
for j, (bkg_keV, bkg_ch) in enumerate(zip(k40_energy_roi['bkg_keV'], k40_channels['bkg'])):
    print(f'  Bkg sideband {j+1}: {bkg_keV} keV  ->  channels {bkg_ch}')

print()

# Step 1: Sum counts in peak window (gross counts)
peak_lo, peak_hi = k40_channels['peak']
gross_counts = np.sum(counts[peak_lo:peak_hi])
n_peak_channels = peak_hi - peak_lo
print(f'Step 1: Gross counts in peak window [{peak_lo}:{peak_hi}] = {gross_counts:.0f}')
print(f'        Number of peak channels: {n_peak_channels}')

# Step 2: Sum counts in background sidebands
bkg_total = 0
n_bkg_channels = 0
for bkg in k40_channels['bkg']:
    bkg_sum = np.sum(counts[bkg[0]:bkg[1]])
    n_bkg = bkg[1] - bkg[0]
    bkg_total += bkg_sum
    n_bkg_channels += n_bkg
    print(f'Step 2: Bkg sideband [{bkg[0]}:{bkg[1]}]: {bkg_sum:.0f} counts in {n_bkg} channels')

# Step 3: Background per channel
bkg_per_channel = bkg_total / n_bkg_channels if n_bkg_channels > 0 else 0
print(f'Step 3: Background per channel = {bkg_total:.0f} / {n_bkg_channels} = {bkg_per_channel:.2f}')

# Step 4: Estimated background under peak
bkg_under_peak = bkg_per_channel * n_peak_channels
print(f'Step 4: Estimated background under peak = {bkg_per_channel:.2f} * {n_peak_channels} = {bkg_under_peak:.2f}')

# Step 5: Net counts
net_counts = gross_counts - bkg_under_peak
print(f'Step 5: Net counts = {gross_counts:.0f} - {bkg_under_peak:.2f} = {net_counts:.2f}')

# Step 6: Error
error = np.sqrt(gross_counts + bkg_under_peak)
print(f'Step 6: Error = sqrt({gross_counts:.0f} + {bkg_under_peak:.2f}) = {error:.2f}')

print()
# Compare with ROI.get_counts()
k40_roi_obj = [r for r in rois if r.isotope == 'K40'][0]
roi_counts, roi_error = k40_roi_obj.get_counts(counts)
print(f'ROI.get_counts() result:  counts = {roi_counts:.2f}, error = {roi_error:.2f}')
print(f'Manual calculation:       counts = {net_counts:.2f}, error = {error:.2f}')
print(f'(Differences are expected since the energy-based ROI may use slightly different channels.)')

In [ ]:
# Zoomed view of K-40 peak region
fig, ax = plt.subplots(figsize=(12, 5))

# Plot range
all_ch = [k40_channels['peak'][0], k40_channels['peak'][1]]
for bkg in k40_channels['bkg']:
    all_ch.extend(bkg)
plot_lo = min(all_ch) - 20
plot_hi = max(all_ch) + 20
plot_lo = max(0, plot_lo)
plot_hi = min(n_channels - 1, plot_hi)

ch_range = np.arange(plot_lo, plot_hi + 1)
e_range = c0 + c1 * ch_range

ax.plot(e_range, counts[plot_lo:plot_hi + 1], 'b-', linewidth=1.2, label='Spectrum')

# Shade peak window
peak_e_lo = c0 + c1 * peak_lo
peak_e_hi = c0 + c1 * peak_hi
ax.axvspan(peak_e_lo, peak_e_hi, alpha=0.3, color='green', label='Peak window')

# Shade background sidebands
for j, bkg in enumerate(k40_channels['bkg']):
    bkg_e_lo = c0 + c1 * bkg[0]
    bkg_e_hi = c0 + c1 * bkg[1]
    ax.axvspan(bkg_e_lo, bkg_e_hi, alpha=0.15, color='gray', hatch='///',
               label='Background' if j == 0 else None)

# Background level line
ax.axhline(bkg_per_channel, color='red', linestyle='--', alpha=0.7,
           label=f'Bkg level ({bkg_per_channel:.1f} cts/ch)')

ax.axvline(1459.41, color='orange', linestyle=':', alpha=0.8, label='Expected 1459.41 keV')

ax.set_xlabel('Energy (keV)')
ax.set_ylabel('Counts')
ax.set_title(f'K-40 Peak: Net counts = {net_counts:.1f} +/- {error:.1f}')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Section 8: Compare Channel-Based vs Energy-Based Results

Run both approaches on the same spectrum for all isotopes.

In [ ]:
print(f'{"Isotope":<10} {"Ch-based counts":<18} {"E-based counts":<18} {"% Diff":<10} {"Ch-based err":<14} {"E-based err"}')
print('-' * 90)

for roi_obj, e_roi in zip(rois, energy_rois):
    # Channel-based (existing)
    ch_counts, ch_error = roi_obj.get_counts(counts)
    
    # Energy-based
    ch_def = energy_roi_to_channels(e_roi, cal)
    e_roi_obj = sample_collection.ROI(ch_def['peak'], ch_def['bkg'])
    e_counts, e_error = e_roi_obj.get_counts(counts)
    
    # Percent difference
    if ch_counts != 0:
        pct_diff = (e_counts - ch_counts) / abs(ch_counts) * 100
    else:
        pct_diff = float('nan')
    
    print(f'{roi_obj.isotope:<10} {ch_counts:<18.2f} {e_counts:<18.2f} {pct_diff:<+10.1f} {ch_error:<14.2f} {e_error:.2f}')

In [ ]:
# Visual comparison: Energy-based ROI overlays on spectrum
# Same layout as Section 4 but using energy-based ROI windows

colors = plt.cm.Set1(np.linspace(0, 1, len(energy_rois)))

fig, axes = plt.subplots(len(energy_rois), 1, figsize=(14, 4 * len(energy_rois)))

for i, e_roi in enumerate(energy_rois):
    ax = axes[i]
    ch_def = energy_roi_to_channels(e_roi, cal)
    
    # Determine plot range with margin
    all_channels = [ch_def['peak'][0], ch_def['peak'][1]]
    for bkg in ch_def['bkg']:
        all_channels.extend(bkg)
    ch_lo = max(0, min(all_channels) - 30)
    ch_hi = min(n_channels - 1, max(all_channels) + 30)
    
    e_lo = c0 + c1 * ch_lo
    e_hi = c0 + c1 * ch_hi
    
    # Plot spectrum in this range
    mask = (energies >= e_lo) & (energies <= e_hi)
    ax.plot(energies[mask], counts[mask], 'b-', linewidth=1)
    
    # Shade energy-based peak window (solid)
    ax.axvspan(e_roi['peak_keV'][0], e_roi['peak_keV'][1],
               alpha=0.3, color=colors[i], label='Energy-based peak window')
    
    # Shade energy-based background sidebands (hatched)
    for j, (bkg_lo, bkg_hi) in enumerate(e_roi['bkg_keV']):
        ax.axvspan(bkg_lo, bkg_hi, alpha=0.15, color='gray',
                   hatch='///', label='Energy-based bkg' if j == 0 else None)
    
    # Also show the OLD channel-based window as a dashed outline for comparison
    if i < len(rois):
        r = rois[i]
        old_peak_lo = c0 + c1 * r.peak[0]
        old_peak_hi = c0 + c1 * r.peak[1]
        ax.axvspan(old_peak_lo, old_peak_hi, alpha=0, edgecolor='red',
                   linewidth=2, linestyle='--', label='Old channel-based peak')
    
    # Mark expected energy
    ax.axvline(e_roi['energy'], color='red', linestyle=':', alpha=0.7,
               label=f'Expected {e_roi["energy"]} keV')
    
    ax.set_xlabel('Energy (keV)')
    ax.set_ylabel('Counts')
    ax.set_title(f'{e_roi["isotope"]} ({e_roi["energy"]} keV) - {e_roi["origin"]}')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(True, alpha=0.3)

plt.suptitle('Energy-Based ROI Windows (solid) vs Old Channel-Based (dashed red outline)',
             fontsize=13, y=1.001)
plt.tight_layout()
plt.show()

## Section 9: Calibration Drift Robustness Demo

Simulate a 2 keV offset shift in calibration and show that:
- Channel-based ROIs give wrong results (still using old channels)
- Energy-based ROIs auto-correct (recompute channels from new calibration)

In [ ]:
# Simulate a shifted calibration (2 keV offset)
cal_shifted = [c0 + 2.0, c1, c2, c3]
print(f'Original calibration: c0 = {c0:.6f}')
print(f'Shifted calibration:  c0 = {cal_shifted[0]:.6f}  (+2 keV offset)')
print()

print(f'{"Isotope":<10} {"Original(ch-based)":<20} {"Shifted(ch-based)":<20} {"Shifted(E-based)":<20}')
print('-' * 75)

for roi_obj, e_roi in zip(rois, energy_rois):
    # Original channel-based result
    original_counts, _ = roi_obj.get_counts(counts)
    
    # Channel-based with shifted calibration: channels DON'T change
    # (this is the problem — same channels, wrong energies)
    ch_shifted_counts = original_counts  # channels are hardcoded, no change
    
    # Energy-based with shifted calibration: channels auto-update
    ch_def_shifted = energy_roi_to_channels(e_roi, cal_shifted)
    e_roi_shifted = sample_collection.ROI(ch_def_shifted['peak'], ch_def_shifted['bkg'])
    e_shifted_counts, _ = e_roi_shifted.get_counts(counts)
    
    print(f'{roi_obj.isotope:<10} {original_counts:<20.2f} {ch_shifted_counts:<20.2f} {e_shifted_counts:<20.2f}')

print()
print('Channel-based ROIs are UNCHANGED despite calibration shift (they use hardcoded channels).')
print('Energy-based ROIs ADAPT by recomputing channel boundaries from the new calibration.')
print('If the calibration drift is real, the energy-based approach gives correct results.')

## Section 10: Time Series and Data Gap Visualization

Load the HDF5 rebinned data and demonstrate the NaN masking fix for data gaps.

**Note:** The HDF5 file contains already-processed (rebinned) data — 1-hour summed spectra
with averaged weather. It must be regenerated on the remote system before this section works.
The notebook will check if the HDF5 has valid data and skip gracefully if empty.

In [ ]:
h5_path = '../rebin.h5'

if not os.path.exists(h5_path):
    print(f'HDF5 file not found at {h5_path}. Skipping time series analysis.')
    print('This file must be generated on the remote system by running the data pipeline.')
    HAS_H5_DATA = False
else:
    try:
        h5file = h5py.File(h5_path, 'r')
        
        # Support both 'data' group (new) and legacy year groups
        if 'data' in h5file:
            data_group = h5file['data']
        else:
            available_groups = list(h5file.keys())
            if len(available_groups) == 0:
                print('HDF5 file has no data groups. Skipping.')
                HAS_H5_DATA = False
                h5file.close()
            else:
                data_group = h5file[available_groups[0]]
        
        required = ['timestamps', 'spectra_meta', 'spectra', 'weather_data']
        missing = [ds for ds in required if ds not in data_group]
        if missing:
            print(f'HDF5 missing datasets: {missing}. Skipping.')
            HAS_H5_DATA = False
            h5file.close()
        else:
            h5_timestamps = data_group['timestamps'][:]
            h5_spectra = data_group['spectra'][:]
            h5_tm_meta = data_group['spectra_meta'][:]
            h5_weather = data_group['weather_data'][:]
            h5file.close()
            
            if len(h5_timestamps) == 0:
                print('HDF5 has no data entries. Skipping.')
                HAS_H5_DATA = False
            else:
                HAS_H5_DATA = True
                print(f'Loaded {len(h5_timestamps)} time bins from HDF5')
                print(f'Spectra shape: {h5_spectra.shape}')
    except Exception as e:
        print(f'Error loading HDF5: {e}')
        HAS_H5_DATA = False

In [ ]:
import datetime

if HAS_H5_DATA:
    # Build ROI array the same way h5_analysis.py does
    col = sample_collection.SampleCollection()
    col.add_roi('analysis/roi_simple.dat')
    
    n_times = h5_spectra.shape[0]
    n_rois = len(col.rois)
    
    roi_array = np.zeros((n_times, n_rois, 2))
    for x in range(n_times):
        tmp = np.zeros((n_rois, 2))
        k = 0
        for el in col.rois:
            live_time = h5_tm_meta[x, 1]
            if live_time > 0:
                tmp[k] = np.array(el.get_counts(h5_spectra[x, :])) / live_time
            k += 1
        roi_array[x, :, :] = tmp
    
    # Convert timestamps to datetime
    datetimes = [datetime.datetime.utcfromtimestamp(t) for t in h5_timestamps]
    
    # --- Plot WITH zeros (before fix) ---
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    # K40 is index 4 in the roi list
    k40_idx = next(i for i, r in enumerate(col.rois) if r.isotope == 'K40')
    
    ax1.plot(datetimes, roi_array[:, k40_idx, 0], 'b-', linewidth=0.8)
    ax1.set_ylabel('Count rate (cps)')
    ax1.set_title('K-40 count rate — BEFORE NaN fix (zeros visible at gaps)')
    ax1.grid(True, alpha=0.3)
    
    # --- Apply NaN mask (the fix) ---
    roi_array_fixed = roi_array.copy()
    for x in range(n_times):
        if h5_tm_meta[x, 1] <= 0 or np.sum(h5_spectra[x, :]) == 0:
            roi_array_fixed[x, :, :] = np.nan
    
    ax2.plot(datetimes, roi_array_fixed[:, k40_idx, 0], 'b-', linewidth=0.8)
    ax2.set_ylabel('Count rate (cps)')
    ax2.set_xlabel('Time (UTC)')
    ax2.set_title('K-40 count rate — AFTER NaN fix (gaps shown as line breaks)')
    ax2.grid(True, alpha=0.3)
    
    fig.autofmt_xdate()
    plt.tight_layout()
    plt.show()
    
    n_nan = np.sum(np.isnan(roi_array_fixed[:, 0, 0]))
    print(f'Total time bins: {n_times}')
    print(f'Invalid bins masked with NaN: {n_nan}')
    print(f'Valid bins: {n_times - n_nan}')
else:
    print('Skipping time series visualization (no HDF5 data available).')

## Section 11: Proposed Energy ROI File Format

Define a new text format (`roi_energy.dat`) with keV-based windows,
plus generator and parser functions.

In [ ]:
def write_energy_roi_file(filepath, energy_rois):
    """Write energy-based ROI definitions to a text file.
    
    Format:
        $ROI_ENERGY:
        <n_bkg> <peak_lo_keV> <peak_hi_keV> <isotope> <energy>keV <origin>
        - <bkg_lo_keV> <bkg_hi_keV>
        ...
    """
    with open(filepath, 'w') as f:
        f.write('$ROI_ENERGY:\n')
        for roi in energy_rois:
            n_bkg = len(roi['bkg_keV'])
            origin_str = roi['origin'].replace(' ', '_')
            f.write(f"{n_bkg} {roi['peak_keV'][0]:.1f} {roi['peak_keV'][1]:.1f} "
                    f"{roi['isotope']} {roi['energy']}keV {origin_str}\n")
            for bkg_lo, bkg_hi in roi['bkg_keV']:
                f.write(f"- {bkg_lo:.1f} {bkg_hi:.1f}\n")
    print(f'Wrote {len(energy_rois)} energy ROIs to {filepath}')


def read_energy_roi_file(filepath):
    """Read energy-based ROI definitions from text file.
    Returns a list of dicts matching the energy_rois format."""
    roi_list = []
    with open(filepath) as f:
        lines = f.readlines()
    
    k = 0
    # Skip to header
    while k < len(lines) and '$ROI_ENERGY' not in lines[k]:
        k += 1
    k += 1
    
    while k < len(lines):
        line = lines[k].strip()
        if not line or line.startswith('#'):
            k += 1
            continue
        parts = line.split()
        n_bkg = int(parts[0])
        peak_lo = float(parts[1])
        peak_hi = float(parts[2])
        isotope = parts[3]
        energy = float(parts[4].replace('keV', ''))
        origin = parts[5].replace('_', ' ') if len(parts) > 5 else ''
        
        bkg_list = []
        for _ in range(n_bkg):
            k += 1
            bkg_parts = lines[k].strip().split()
            bkg_list.append((float(bkg_parts[1]), float(bkg_parts[2])))
        
        roi_list.append({
            'isotope': isotope,
            'energy': energy,
            'peak_keV': (peak_lo, peak_hi),
            'bkg_keV': bkg_list,
            'origin': origin,
        })
        k += 1
    
    return roi_list


# Write and read back to verify
write_energy_roi_file('analysis/roi_energy.dat', energy_rois)

# Read it back
loaded = read_energy_roi_file('analysis/roi_energy.dat')
print(f'\nRead back {len(loaded)} ROIs:')
for roi in loaded:
    print(f"  {roi['isotope']}: {roi['energy']} keV, peak {roi['peak_keV']}, "
          f"{len(roi['bkg_keV'])} bkg regions")

# Verify round-trip
print('\nRound-trip verification:')
for orig, loaded_roi in zip(energy_rois, loaded):
    match = (orig['isotope'] == loaded_roi['isotope'] and
             orig['peak_keV'] == loaded_roi['peak_keV'] and
             len(orig['bkg_keV']) == len(loaded_roi['bkg_keV']))
    print(f"  {orig['isotope']}: {'OK' if match else 'MISMATCH'}")

---

## Summary

This notebook has:
1. Loaded and verified the calibration (single source of truth)
2. Visually verified ROI alignment on a real spectrum
3. Defined energy-based ROI alternatives
4. Demonstrated step-by-step net count extraction
5. Compared channel-based vs energy-based results
6. Shown robustness to calibration drift
7. Demonstrated the NaN fix for data gaps in time series
8. Proposed and tested a new energy-based ROI file format

### Future Work
- Add `energy_to_channel()` to `spectrum_calibration.py`
- Add `parse_roi_energy()` to `spectra_utils.py`
- Create production `roi_energy.dat` file
- Update `ROI.get_counts()` to accept calibration and work in energy space
- Update `h5_analysis.py` to use energy-based ROIs